# 如何使用MultiQueryRetriever
基于距离的[向量数据库](/docs/concepts/vectorstores/)检索通过将查询[嵌入](/docs/concepts/embedding_models/)（表示）到高维空间中，并根据距离度量找到相似的嵌入文档。然而，检索结果可能会因查询措辞的细微变化，或嵌入未能很好地捕捉数据语义而产生差异。虽然可以通过提示工程/调优手动解决这些问题，但这一过程往往较为繁琐。
[MultiQueryRetriever](https://python.langchain.com/api_reference/langchain/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html) 通过使用大语言模型（LLM）为给定的用户输入查询生成多个不同角度的查询，从而自动化提示调优过程。对于每个生成的查询，它会检索一组相关文档，并取所有查询结果的唯一并集，以获得更广泛的潜在相关文档集合。通过对同一问题生成多视角的查询，`MultiQueryRetriever` 能够缓解基于距离的检索方法的某些局限性，从而得到更丰富的结果集。
让我们根据[RAG教程](/docs/tutorials/rag)中Lilian Weng的博文[LLM驱动的自主代理](https://lilianweng.github.io/posts/2023-06-23-agent/)，构建一个向量数据库。

In [1]:
# Build a sample vectorDB
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load blog post
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)

# VectorDB
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)

USER_AGENT environment variable not set, consider setting it to identify your requests.


#### 简单用法
指定用于查询生成的LLM，检索器将完成其余工作。

In [2]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

question = "What are the approaches to Task Decomposition?"
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)

In [3]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [4]:
unique_docs = retriever_from_llm.invoke(question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. How can Task Decomposition be achieved through different methods?', '2. What strategies are commonly used for Task Decomposition?', '3. What are the various ways to break down tasks in Task Decomposition?']


5

请注意，由[检索器](/docs/concepts/retrievers/)生成的基础查询会以`INFO`级别记录在日志中。

#### 提供您自己的提示
在底层实现中，`MultiQueryRetriever` 通过特定的[提示模板](https://python.langchain.com/api_reference/langchain/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html)生成查询。如需自定义该提示模板：
1. 创建一个带有问题输入变量的[PromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html);2. 实现一个如下所示的[输出解析器](/docs/concepts/output_parsers)，将结果拆分为查询列表。
提示词与输出解析器必须共同支持生成查询列表。

In [5]:
from typing import List

from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field


# Output parser will split the LLM result into a list of queries
class LineListOutputParser(BaseOutputParser[List[str]]):
    """Output parser for a list of lines."""

    def parse(self, text: str) -> List[str]:
        lines = text.strip().split("\n")
        return list(filter(None, lines))  # Remove empty lines


output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions separated by newlines.
    Original question: {question}""",
)
llm = ChatOpenAI(temperature=0)

# Chain
llm_chain = QUERY_PROMPT | llm | output_parser

# Other inputs
question = "What are the approaches to Task Decomposition?"

In [6]:
# Run
retriever = MultiQueryRetriever(
    retriever=vectordb.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

# Results
unique_docs = retriever.invoke("What does the course say about regression?")
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide insights on regression from the course material?', '2. How is regression discussed in the course content?', '3. What information does the course offer regarding regression?', '4. In what way is regression covered in the course?', "5. What are the course's teachings on regression?"]


9